In [3]:
import random
import torch
from d2l import torch as d2l

In [4]:
# 生成数据集
def synthetic_data(w, b, num_examples):
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

In [5]:
# 读取数据集
# 定义一个函数，打乱数据集中的样本并以小批量方式获取数据
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(indices[i:min(i+batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]

batch_size = 10

for X, y in data_iter(batch_size, features, labels):
    print(X, '\n', y)
    break

tensor([[ 0.1982,  0.1541],
        [ 0.1559, -0.1716],
        [-0.1612, -1.5772],
        [ 0.8802,  1.1479],
        [-1.5781, -0.7898],
        [ 0.3515,  0.1289],
        [ 2.4794,  0.2406],
        [ 0.6460,  0.4554],
        [-0.9447, -0.9094],
        [-0.7813, -0.6094]]) 
 tensor([[4.0588],
        [5.0942],
        [9.2420],
        [2.0594],
        [3.7307],
        [4.4648],
        [8.3316],
        [3.9503],
        [5.4054],
        [4.7070]])


In [6]:
# 初始化模型参数
# 通过从均值为0、标准差为0.01的正态分布中采样随机数来初始化权重，并将偏置初始化为0
w = torch.normal(0, 0.01, size=(2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# 初始化参数之后，接下来的任务是更新这些参数，直到这些参数足够拟合数据
# 每次更新都需要计算损失函数关于模型参数的梯度

In [7]:
# 定义模型
# 定义模型，将模型的输入和参数同模型的输出关联起来
def linreg(X, w, b):
    '''线性回归模型'''
    return torch.matmul(X, w) + b

In [8]:
# 定义损失函数
def squared_loss(y_hat, y):
    '''均方损失'''
    return (y_hat - y.reshape(y_hat.shape))**2/2

In [9]:
# 定义优化算法
def sgd(params, lr, batch_size):
    '''小批量随机梯度下降'''
    with torch.no_grad():
        for param in params:
            param -= lr*param.grad/batch_size
            param.grad.zero_()

In [ ]:
# 训练
lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y)
        l.sum().backward()
        sgd([w, b], lr, batch_size)
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)
        print(f'epoch{epoch + 1}, loss{float(train_l.mean()):f}')